## Implementation of Feed Forward Neural Network for Fashion_mnist dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import mnist, fashion_mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")
import wandb
import seaborn as sns

### Importing and Preprocessing of fashion_mnist dataset

In [2]:
from keras.datasets import fashion_mnist


In [3]:
(X,Y),(X_test,Y_test) = fashion_mnist.load_data()

In [4]:
#reshaping the X to flatten it and then dividing the pixels value by 255 to normalize them.
X = X.reshape(len(X),784,1)
X[0].shape
X_test = X_test.reshape(len(X_test),784,1)
X = X/255.0
X_test = X_test/255.0

In [5]:
X_train,X_val,Y_train,Y_val = train_test_split(X,Y,test_size=0.1,random_state=42)

In [15]:
# Made a class NeuralNetwork that contains all the necessary functions and parameters for a neural network.
# Wrote code for Feedforward that is flexible enough to handle large inputs with variable layer sizes because of
# the way I have handled the parameters initialization. Given the dimensions of each layer of the network allows the code to 
# make the necessary initializations and can be adjusted easily to your like.
class NeuralNetwork:
    def __init__(self,optimizer='nadam',batchsize=32,no_of_features = 784,no_of_classes = 10,no_of_layers=5,no_of_neurons_in_each_layer = [128,128,128],max_epochs = 10,eta = 0.001,initialization_method = 'he',activation_method = 'relu',loss = 'cross',weight_decay = 0.001):
        self.optimizer  = optimizer
        self.batchsize = batchsize
        self.no_of_features = no_of_features
        self.no_of_classes = no_of_classes
        self.no_of_layers = no_of_layers
        self.no_of_neurons_in_each_layer = no_of_neurons_in_each_layer
        self.max_epochs = max_epochs
        self.eta = eta
        self.initialization = initialization_method
        self.initialization_list = {'xavier':self.xavier_initialization,'uniform':self.uniform_initialization,'normal':self.normal_initialization,'he':self.he_initialization}
        self.activation = activation_method
        self.activation_list = {'sigmoid':self.sigmoid,'relu':self.Relu,'tanh':self.tanh}
        self.activation_derivative = {'sigmoid':self.sigmoid_derivative,'relu':self.Relu_derivative,'tanh':self.tanh_derivative}
        self.loss = loss
        self.thetas = {}
        self.loss_list = []
        self.weight_decay = weight_decay
    def one_hot(self,l,no_of_classes):
        temp = np.array([0]*no_of_classes)
        temp[l] = 1
        return temp
    def sigmoid(self,x):
        return 1. / (1.+np.exp(-x))

    def sigmoid_derivative(self,x):
        return self.sigmoid(x) * (np.ones_like(x)-self.sigmoid(x))

    def Relu(self,x):
        for i in range(len(x)):
            x[i] = x[i] / max(x[i])
        return np.maximum(0,x)

    def Relu_derivative(self,x):
        # for i in range(len(x)):
        #     x[i] = x[i] / max(x[i])
        return 1*(x>0) 

    def tanh(self,x):
        # for i in range(len(x)):
        #     x[i] = x[i] / max(x[i])
        return np.tanh(x)

    def tanh_derivative(self,x):
        # for i in range(len(x)):
        #     x[i] = x[i] / max(x[i])
        return (1 - (np.tanh(x)**2))

    def softmax(self,x):
        # for i in range(len(x)):
        #     x[i] = x[i] / max(x[i])
        l = []
        for i in range(len(x)):
            l.append(np.exp(x[i])/np.sum(np.exp(x[i]),axis=0))
        return np.array(l)
        
    def softmax_derivative(self,x):
        # for i in range(len(x)):
        #     x[i] = x[i] / max(x)
        return self.softmax(x.reshape(1,x.shape[0],x.shape[1])) * (1-self.softmax(x.reshape(1,x.shape[0],x.shape[1])))



    def he_initialization(self):
        np.random.seed(42)
        thetas = {}
        for layer in range(1,self.no_of_layers):
            if(layer == 1):
                thetas['W'+str(layer)] = np.random.normal(0,1,size = (self.no_of_neurons_in_each_layer[layer-1],self.no_of_features)) * np.sqrt(2/(self.no_of_neurons_in_each_layer[layer-1]))
                thetas['b'+str(layer)] = np.random.normal(0,1,size = (self.no_of_neurons_in_each_layer[layer-1],1))*np.sqrt(2/self.no_of_neurons_in_each_layer[layer-1])
            elif(layer == self.no_of_layers-1):
                thetas['W'+str(layer)] = np.random.normal(0,1,size = (self.no_of_classes,self.no_of_neurons_in_each_layer[layer-2])) * np.sqrt(2/(self.no_of_classes))
                thetas['b'+str(layer)] = np.random.normal(0,1,size = (self.no_of_classes,1)) * np.sqrt(2/(self.no_of_classes))
            else:
                thetas['W'+str(layer)] = np.random.normal(0,1,size = (self.no_of_neurons_in_each_layer[layer-1],self.no_of_neurons_in_each_layer[layer-2])) * np.sqrt(2/(self.no_of_neurons_in_each_layer[layer-1]))
                thetas['b'+str(layer)] = np.random.normal(0,1,size = (self.no_of_neurons_in_each_layer[layer-1],1)) * np.sqrt(2/self.no_of_neurons_in_each_layer[layer-1])
        return thetas

    def xavier_initialization(self):
        np.random.seed(42)
        thetas = {}
        for layer in range(1,self.no_of_layers):
            if(layer == 1):
                thetas['W'+str(layer)] = np.random.randn(self.no_of_neurons_in_each_layer[layer-1],self.no_of_features) * np.sqrt(2/(self.no_of_neurons_in_each_layer[layer-1]+self.no_of_features))
                thetas['b'+str(layer)] = np.zeros((self.no_of_neurons_in_each_layer[layer-1],1))
            elif(layer == self.no_of_layers-1):
                thetas['W'+str(layer)] = np.random.randn(self.no_of_classes,self.no_of_neurons_in_each_layer[layer-2]) * np.sqrt(2/(self.no_of_classes + self.no_of_neurons_in_each_layer[layer-2]))
                thetas['b'+str(layer)] = np.zeros((self.no_of_classes,1))
            else:
                # print(layer)
                # print(self.no_of_neurons_in_each_layer[layer-1])
                # print(self.no_of_neurons_in_each_layer[layer-2])
                thetas['W'+str(layer)] = np.random.randn(self.no_of_neurons_in_each_layer[layer-1],self.no_of_neurons_in_each_layer[layer-2]) * np.sqrt(2/(self.no_of_neurons_in_each_layer[layer-1]+self.no_of_neurons_in_each_layer[layer-2]))
                thetas['b'+str(layer)] = np.zeros((self.no_of_neurons_in_each_layer[layer-1],1))
        return thetas

    def uniform_initialization(self):
        thetas = {}
        np.random.seed(42)
        for layer in range(1,self.no_of_layers):
            if(layer == 1):
                thetas['W'+str(layer)] = np.random.default_rng().uniform(low = -0.7,high =0.7,size = (self.no_of_neurons_in_each_layer[layer-1],self.no_of_features)) #* np.sqrt(2/(no_of_neurons_in_each_layer[layer-1]+no_of_features))
                thetas['b'+str(layer)] = np.random.default_rng().uniform(low = -0.7,high =0.7,size = (self.no_of_neurons_in_each_layer[layer-1],1))
            elif(layer == self.no_of_layers-1):
                thetas['W'+str(layer)] = np.random.default_rng().uniform(low = -0.7,high =0.7,size = (self.no_of_classes,self.no_of_neurons_in_each_layer[layer-2])) #* np.sqrt(2/(no_of_classes + no_of_neurons_in_each_layer[layer-2]))
                thetas['b'+str(layer)] = np.random.default_rng().uniform(low = -0.7,high =0.7,size = (self.no_of_classes,1))
            else:
                thetas['W'+str(layer)] = np.random.default_rng().uniform(low = -0.7,high =0.7,size =(self.no_of_neurons_in_each_layer[layer-1],self.no_of_neurons_in_each_layer[layer-2])) #*  np.sqrt(2/(no_of_neurons_in_each_layer[layer-1]+no_of_neurons_in_each_layer[layer-2]))
                thetas['b'+str(layer)] = np.random.default_rng().uniform(low = -0.7,high =0.7,size = (self.no_of_neurons_in_each_layer[layer-1],1))
        return thetas

    def normal_initialization(self):
        thetas = {}
        np.random.seed(42)
        for layer in range(1,self.no_of_layers):
            if(layer == 1):
                thetas['W'+str(layer)] = np.random.uniform(low = -0.7,high =0.7,size = (self.no_of_neurons_in_each_layer[layer-1],self.no_of_features)) #* np.sqrt(2/(no_of_neurons_in_each_layer[layer-1]+no_of_features))
                thetas['b'+str(layer)] = np.zeros((self.no_of_neurons_in_each_layer[layer-1],1))
            elif(layer == self.no_of_layers-1):
                thetas['W'+str(layer)] = np.random.uniform(low = -0.7,high =0.7,size = (self.no_of_classes,self.no_of_neurons_in_each_layer[layer-2])) #* np.sqrt(2/(no_of_classes + no_of_neurons_in_each_layer[layer-2]))
                thetas['b'+str(layer)] = np.zeros((self.no_of_classes,1))
            else:
                thetas['W'+str(layer)] = np.random.uniform(low = -0.7,high =0.7,size =(self.no_of_neurons_in_each_layer[layer-1],self.no_of_neurons_in_each_layer[layer-2])) #*  np.sqrt(2/(no_of_neurons_in_each_layer[layer-1]+no_of_neurons_in_each_layer[layer-2]))
                thetas['b'+str(layer)] = np.zeros((self.no_of_neurons_in_each_layer[layer-1],1))
        return thetas

    def feed_forward(self,data,thetas,layers):
        pre_activation = [1]*(layers)
        activation  = [1]*(layers)
        activation[0] = data
        for layer_no in range(1,layers):
            W = 'W' + str(layer_no)
            b = 'b' + str(layer_no)
            pre_activation[layer_no] = np.add(np.matmul(thetas[W],activation[layer_no - 1]),thetas[b])
            if(layer_no == layers-1):
                activation[layer_no] = self.softmax(pre_activation[layer_no])
            else:
                activation[layer_no] = self.activation_list[self.activation](pre_activation[layer_no])
        return activation,pre_activation

    def back_propagate(self,h,a,thetas,Y):
        grads = {}
        for x in thetas.keys():
            grads[x] = 0
        for x in range(len(Y)):
            temp = h[-1][x] - self.one_hot(Y[x],self.no_of_classes).reshape(self.no_of_classes,1)
            if(self.loss == 'mse'):
                temp = (temp*self.softmax_derivative(a[-1][x])).reshape(self.no_of_classes,1)
            for k in range(self.no_of_layers-1,0,-1):
                W = 'W' + str(k)
                b = 'b' + str(k)
                grads[W] += np.matmul(temp,h[k-1][x].T)/self.batchsize
                grads[b] += temp/self.batchsize
                if(k == 1):
                    break
                temp = np.matmul(thetas[W].T,temp)
                temp = np.multiply(temp,self.activation_derivative[self.activation](a[k-1][x]))
        return grads

    def momentumUpdate(self,t,maxm=.999):
        x=np.log(np.floor(t/250)+1)/np.log(2)
        x=1-2**(-1-x)
        return min(x,maxm)

    def getGamma(self,epoch):
        x=np.log((epoch/250)+1)
        x=-1-1*(x)
        x=2**x
        x=1-x
        return min(x,.9)

    def fit(self,X_train,Y_train):
        self.thetas = self.initialization_list[self.initialization]()
        delta = 1e-9
        grads = {}
        for i in self.thetas.keys():
            grads[i] = 0
        for t in range(self.max_epochs):
            #previous_update
            ut = {}
            vt = {}
            gamma = self.getGamma(t+1)
            beta = self.momentumUpdate(t+1)
            for i in self.thetas.keys():
                ut[i] = 0
                vt[i] = 0
            params_look_ahead = {}
            step = 1
            for x in range(0,X_train.shape[0],self.batchsize):
                beta1 = 0.9#self.momentumUpdate(step)
                beta2 = 0.99#self.momentumUpdate(step)
                if(self.optimizer == 'nesterov'):
                    for i in self.thetas.keys():
                        params_look_ahead[i] = self.thetas[i] - beta1*ut[i]
                    activation,preactivation = self.feed_forward(X_train[x:x+self.batchsize],self.thetas,self.no_of_layers)
                    grads = self.back_propagate(activation,preactivation,params_look_ahead,Y_train[x:x+self.batchsize])
                    for i in self.thetas.keys():
                        ut[i] = beta1*ut[i] + (1-beta1)*grads[i]
                        self.thetas[i] = self.thetas[i] - self.eta*ut[i] - self.eta*self.weight_decay*self.thetas[i]
                elif(self.optimizer == 'mgd'):
                    activation,preactivation = self.feed_forward(X_train[x:x+self.batchsize],self.thetas,self.no_of_layers)
                    grads = self.back_propagate(activation,preactivation,self.thetas,Y_train[x:x+self.batchsize])     
                    for i in self.thetas.keys():
                        ut[i] = gamma*ut[i] + grads[i]
                        self.thetas[i] = self.thetas[i] - self.eta*ut[i] - self.eta*self.weight_decay*self.thetas[i]
                elif(self.optimizer == 'sgd'):
                    activation,preactivation = self.feed_forward(X_train[x:x+self.batchsize],self.thetas,self.no_of_layers)
                    grads = self.back_propagate(activation,preactivation,self.thetas,Y_train[x:x+self.batchsize])
                    for i in self.thetas.keys():
                        self.thetas[i] = self.thetas[i] - self.eta*grads[i] - self.eta*self.weight_decay*self.thetas[i]
                elif(self.optimizer == 'RMSprop'):
                    activation,preactivation = self.feed_forward(X_train[x:x+self.batchsize],self.thetas,self.no_of_layers)
                    grads = self.back_propagate(activation,preactivation,self.thetas,Y_train[x:x+self.batchsize])
                    for i in self.thetas.keys():
                        ut[i] = beta*ut[i] + (1-beta)*np.multiply(grads[i],grads[i])
                        self.thetas[i] = self.thetas[i] - self.eta*grads[i]/((np.sqrt(ut[i])+delta)) - self.eta*self.weight_decay*self.thetas[i]
                elif(self.optimizer == 'adam'):
                    activation,preactivation = self.feed_forward(X_train[x:x+self.batchsize],self.thetas,self.no_of_layers)
                    grads = self.back_propagate(activation,preactivation,self.thetas,Y_train[x:x+self.batchsize])
                    for i in self.thetas.keys():
                        ut[i] = beta1*ut[i] + (1-beta1)*grads[i]
                        uthat = ut[i]/(1 - pow(beta1,t+1))
                        vt[i] = beta2*vt[i] + (1-beta2)*np.multiply(grads[i],grads[i])
                        vthat = vt[i]/(1 - pow(beta2,t+1))
                        self.thetas[i] = self.thetas[i] - self.eta*uthat/((np.sqrt(vthat) + delta)) - self.eta*self.weight_decay*self.thetas[i]
                elif(self.optimizer == 'nadam'):
                    activation,preactivation = self.feed_forward(X_train[x:x+self.batchsize],self.thetas,self.no_of_layers)
                    grads = self.back_propagate(activation,preactivation,self.thetas,Y_train[x:x+self.batchsize])
                    for i in self.thetas.keys():
                        ut[i] = beta1*ut[i] + (1-beta1)*grads[i]
                        uthat = ut[i]/(1 - pow(beta1,t+1))
                        vt[i] = beta2*vt[i] + (1-beta2)*np.multiply(grads[i],grads[i])
                        vthat = vt[i]/(1 - pow(beta2,t+1))
                        self.thetas[i] = self.thetas[i] - (self.eta*(beta1*uthat + (1-beta1)*grads[i]/(1-pow(beta1,t+1))))/(np.sqrt(vthat) + delta) - self.eta*self.weight_decay*self.thetas[i]
                step+=1
            yhat = self.predict(X_train)
            training_accuracy = self.accuracy_score(Y_train,yhat)
            # print(training_accuracy)
            training_loss = self.calculateLoss(yhat,Y_train)
            # print(training_loss)
            self.loss_list.append(training_loss)
            yhat = self.predict(X_val)
            validation_accuracy = self.accuracy_score(Y_val,yhat)
            # print(validation_accuracy)
            validation_loss = self.calculateLoss(yhat,Y_val)
            # print(validation_loss)
            #wandb.log({'training_accuracy' : training_accuracy, 'validation_accuracy' : validation_accuracy,'training_loss' : training_loss, 'validation_loss' : validation_loss,'epoch':t+1})
                
    def predict(self,X):
            activation,preactivation = self.feed_forward(X[:],self.thetas,self.no_of_layers)
            return activation[-1]

    def accuracy_score(self,Y,yhat):
        correct = 0
        for x in range(len(yhat)):
            if(np.argmax(yhat[x]) == Y[x]):
                correct+=1
        return (correct/len(Y)*100)
    def calculateLoss(self,yHat,yBatch):
        loss=0
        l2=0
        if(self.loss == 'cross'):
            for x in range(len(yHat)):
                loss += (-1)*np.log(yHat[x][yBatch[x]] + 1e-9)
            for x in self.thetas.keys():
                l2 += np.linalg.norm(self.thetas[x])
            l2 = (self.weight_decay*l2)/2
            return (loss + l2)/len(yHat)
        if(self.loss == 'mse'):
                # error = (yHat-yBatch)
                # error=error**2
                # loss = np.sum(error,axis=0) 
                # loss = np.sum(error)  
                # loss = loss/2
            for x in range(len(yHat)):
                loss += np.sum((self.one_hot(yBatch[x],self.no_of_classes).reshape(NN.no_of_classes,1) - yHat[x])**2,axis = 0)
            for x in self.thetas.keys():
                l2 += np.linalg.norm(self.thetas[x])
            l2 = (self.weight_decay*l2)/2
            return (loss[0] + l2)/len(yHat)
        return loss

        
        
        

    
    

In [16]:
# Example of usage of feed Forward.
optimizer = 'sgd'
batchsize = 16
no_of_features = 784
no_of_classes = 10
no_of_layers = 5
no_of_neurons_in_each_layer = [128,128,128]
max_epochs = 10
eta = 0.1
initialization_method = 'uniform'
activation_method = 'sigmoid'
weight_decay = 0

In [17]:
NN = NeuralNetwork(optimizer,batchsize,no_of_features,no_of_classes,no_of_layers,no_of_neurons_in_each_layer,max_epochs,eta,initialization_method,activation_method,weight_decay)

In [18]:
NN.fit(X_train,Y_train)

In [ ]:
predicted = NN.predict(X_train)

In [ ]:
print(NN.accuracy_score(Y_train,predicted))